# JSON - Creator

## Import libaries and define dictionary

In [ ]:
from py_src.util import *
import os
import json

# create and structure dictionary
cont = {}
gen = cont['general'] = {}
path = cont['path'] = {}
src = cont['source'] = {}
netw = cont['network'] = {}
pre = cont['preprocessing'] = {}
inv = cont['inversion'] = {}
post = cont['postprocessing'] = {}
log = cont['logger'] = {}
write = cont['writer'] = {}
plot = cont['plotter'] = {}
res = cont['result'] = {}

## Set general path information
Set location of Main.ipynb       

In [ ]:
pwd = '/'.join(os.getcwd().split('/')[:-1])

Path to files (e.g. shared folder).     
Compare structure of Example event.    

In [ ]:
path2fserver = pwd
print(path2fserver)

Set region

In [ ]:
region = 'Example'

## Set event information

#### Access event file

Note: selection in event_reader() affects only the visual output in the notebook (out=True)

In [ ]:
# path to Event_list
Event_file = path2fserver+'/TextFiles/'+region+'/Event_List'
path['Event_file'] = Event_file

# read Event_list
event_list = event_reader(Event_file,out=True,plot=False,
                          depth=[0.,1000.],lat=[-90.,90.],lon=[-180.,180.],mag=[0.,10.])  
src['event_list'] = event_list

# search ID in Event_list by source time
id_cont = get_event_id_from_time(event_list,'2016-10-18T22:08:10.000000Z')
print(id_cont) # output: [ID, time difference to onset time in s]

#### Major centroid information                         
Note: event_depth information can differ from depth information in event file. Set depth to available synthetics depth (see Synthetics below)

In [ ]:
# define region
src['region'] = region

# set event id
src['event_id'] = 'ev123'

# set synthetic event depth
src['event_depth'] = 160

# get source information of selectd id
src['F1_loc'] = event_list[src['event_id']]
print(src['F1_loc'])


#### Minor centroid information (doublet simulation)
Note: event_id and depth can be the same or differ from major couple information.

In [ ]:
src['simulate_doublet'] =  False# True#
src['doublet_event_id'] = src['event_id'] # set id of desired second event at given location
src['doublet_event_depth'] = 160 # set synthetic event depth
src['F2_loc'] = event_list[src['doublet_event_id']] # get source information of selectd id

#### Uncertanty simulation
##### Data covariance (Cd)

In [ ]:
# data covariance 
# sampling uses exponentional factor (see Duputel et. al. 2012)
# simple uses a unity matrix
inv['Cd_modus'] = 'sampling' #'simple'#
# Scaling factor (exponent)
inv['exp_scaling_fac'] = 10**-4 # to avoid nummerical zero with float128. Should be > 0.9*10**-4

##### Model covariance (CT)
Station misalignment

In [ ]:
# solve for station misrotaion (single source only, no mislocation)
# src['solve_for_misloc'] and src['simulate_doublet'] will be set to False
src['solve_for_misrot'] = True #False #  
path['orientation_file'] = path2fserver+'/TextFiles/'+region+'/VOILA_OBS_ORIENTATIONS'

Get orientations and uncertainties.      
Use constant uncertainty angle if orientation_file is missing.    

In [ ]:
src['daz'] = get_orientations(path['orientation_file'])
#src['daz'] = 10. # azimuth uncertainty at station i in degree (az_pert_i = az_i+daz)

Station delay (still in development)         

In [ ]:
# solve for station delay times (single source)
src['solve_for_delay'] = False # True#
src['t_del'] = 3.0 # delay time in s --> currently not used, could be replaced by general station delay info

Mislocation (still in development)

In [ ]:
# solve for mislocation (single source)
src['solve_for_misloc'] = False # True #
src['dx'] = [1,1] # [d(X,Y),dZ] in km
src['misloc_fault_id'] = 'F1' # F1 or F2; solving for mislocation can be done for only one fault 

## Set network information

### Station file

In [ ]:
# path to STATIONS file
Station_file = path2fserver+'/TextFiles/'+region+'/STATIONS_global'
# 
path['Station_file'] = Station_file

Source-Reciever distance range          
Stations outside this range cannot be selected at a later stage (usefull if synthetics are not available)

In [ ]:
netw['distance_range'] = [0,350]

Load stations (optional: display file content and display on map)   
Options:                           
**out**: list content                 
**plot**: display network on map  
**event_id**: give event id to display source location                            
**dist_p**: only if event_id is given. Set two values to display min and max radia or leave emtpy to plot reference circles.

In [ ]:
netw['stat_global'], netw['netw_id'] = station_reader(Station_file,event_list,out=False,plot=True,
                                                      event_id=src['event_id'],dist_p=netw['distance_range'])

## Set data information 
### Observations

Define data_path variable to local observabel database. Set tag out to True to list available data.              
Note: data_path should lead to parant directory with abitary number of subfolders with naming according to event identifier (event_id)

In [ ]:
data_path = path2fserver+'/Data/'+region+'/Observations/'
database_obs = get_event_id_path(data_path,out=True)

Path to observables

In [ ]:
path['observation_data_path'] = database_obs[src['event_id']]

### Synthetics      
We currently support two sources for 1D synthetics and one for 3D synthetics:                   
**CAP**: 1D local database. Simulate waveforms with external simulation codes (e.g. specfem3D) in CAP setting (see below)                                  
**Fomosto**: 1D Greens function database by pyrocko. Codes uses a wrapper to simulate fundamental waveforms from fomosto dictionary.  
**3D**: 3D Greens function database. Please provide MT elementwise pertubations at fixed magnitude for the known source-reciever arangement. Pertubed waveforms should by stored with corresponding ending: .mrr, .mtt, .mpp, .mrt, .mrp, .mtp 

In [ ]:
gen['synt_source'] = 'CAP'# '3D'# 'Pyrocko'#
synt_ref_mag = 5.0 # magnitude of synthetics 

#### CAP

Define data_path variable to local CAP database. Set tag out to True to list available data.             
                  
Note: data_path should lead to parant directory with abitary number of subfolders with naming according to synthetic event depth (km). Each folder is further structured into four subfolders with naming 45DS, 90DS, 90SS and ISO corresponding to the underlying fundamental source mechanism. 

Path to CAP synthetics (set None if CAP is not used)     

In [ ]:
# check availability of CAP waveform database
data_path =  path2fserver+'/Data/'+region+'/Synthetics/'
database_synt_CAP = get_fund_depth_list_CAP(data_path,out=True)

CAP Settings

In [ ]:
path['CAP_synthetic_data_path'] = database_synt_CAP
path['CAP_subfolder_structure'] = ['90SS','90DS','45DS','ISO'] # do not change!
path['CAP_pattern'] = {'90SS':['A','B'],'90DS':['A','C'],'45DS':['B'],'ISO':['B']} # do not change!

# fbd: frequency_band - device id 
netw['cap_fbd_id'] = 'BH' 

# add zero trace in from of CAP-synthetics 
netw['CAP_t_add'] = 60. # in s

# profil settings
netw['profil'] = {'offset':0.0,           # offset from source in km
                  'profil_lenght':350,    # number of receivers in 1
                  'station_sampling':1.0} # inter-station distance in km

# Source-Time function for CAP (this feature needs more tests)
netw['CAP_STF'] = {'F1':0.0,'F2':0.0}
#src['stf'] = {}
#src['stf']['type'] = 'delta' # 
#src['stf']['t_half_1'] = 0.0
#src['stf']['t_half_2'] = 0.0


print(path['CAP_synthetic_data_path'])

#### Fomosto by Pyrocko  ([Info](https://pyrocko.org/docs/current/apps/fomosto))                                     

                                                        
More information about the model can be found at the official [Greens function store](https://greens-mill.pyrocko.org/)                           
                        
                     
     

Searching for a Crust2x2 Profile at central coordinate.          
To download the model (xxx), execute following line in the Terminal at the desired location:                     
$\textbf{fomosto download kinherd xxx}$    

If an update is desired, change settings in config and run:                                  
fomosto ttt --force                  
fomosto build --force  


Path to fomosto Green function database

In [ ]:
import pyrocko.crust2x2
# central location of target area
rlat, rlon = 41.5, 19.5 #  lat, lon (region central point)
profile = pyrocko.crust2x2.get_profile(rlat, rlon) 

# define path
path2store_id = None#path2fserver+'/Data/'+region+'/Synthetics/'
Vmodel = 'ak135_2000km_1Hz'
store_id = [Vmodel]
subnetwork = {Vmodel:list(netw['stat_global'].keys())} # define different subnetworks with different vel models

Fomosto Settings

In [ ]:
# add to fomosto pathinformation to dictionary
path['fomosto_database'] = path2store_id
src['pyrocko_store_id'] = store_id
netw['store_id_subnetwork'] = subnetwork
netw['pyrocko_fbd_id'] = 'HH' # fbd: frequency_band - device id 
netw['trace_length'] = 15.*60. # in s 
netw['Pyrocko_t_add'] = 3*60. # in s 
netw['Pyrocko_STF'] = {'F1':0.0,'F2':0.0}

#### 3D waveform database


Set path to data set identifier list

In [ ]:
path['data_set_identifier'] = None#path2fserver+'/TextFiles/'+region+'/ID_AF_Grid'

Load ID list   
Should contain folder ID and source information:   

set_id              lat    lon  depth reference_Mw ref_moment        
14.95_-61.45_170.0 14.95 -61.45 170.0 7.5 2.00E+20         
14.95_-61.3667_170.0 14.95 -61.3667 170.0 7.5 2.00E+20         
14.95_-61.2833_170.0 14.95 -61.2833 170.0 7.5 2.00E+20         
...

In [ ]:
path['DSI_dict'] = None#load_dataset_identifier(path['data_set_identifier'],out=True)

Path to 3D database

In [ ]:
path['3D_synthetic_data_path'] = None#path2fserver+'/Data/'+region+'/Synthetics/AF_Grid/'

Recomment set_id (returns nearest sythetics to source location)

In [ ]:
LocF1 = src['F1_loc'][1]
path['recommendation_dict_3D'] = None#recomment_set(path['DSI_dict'],path['3D_synthetic_data_path'],
                                      #         LocF1,out=True)

Set set_id     
You can use the recommended set_id or set a custom ID.       
Note: The 3D case ignores all src loction settings set in "Set event information"     

In [ ]:
src['F1_set_id_3d'] = '14.95_-61.45_170.0'
src['F2_set_id_3d'] = '14.95_-61.45_170.0'

Source update      
Overwrite location setting in "Set event information" 

In [ ]:
#src_update = update_3D_source_location(cont)
#src['F1_loc'] = src_update['F1_loc']
#src['F2_loc'] = src_update['F2_loc']
#synt_ref_mag = src_update['ref_mag']
print(src['F1_loc'])

In [ ]:
netw['3D_fbd_id'] = 'BH'
netw['3D_t_add'] = 3*60. # in s 
netw['3D_trace_length'] = 15.*60. # in s
netw['3D_STF'] = {'F1':0.0,'F2':0.0}

### Phase onset information     

#### From cnv-file
Available for cnv picks and fomosto Green function database  

Path to cnv file 

In [ ]:
#path['cnv_file'] = path2fserver+'/TextFiles/'+region+'/500best_events_final.cnv'
path['cnv_file'] = None

Complete the phase pick table by including a event cluster within vicinity of the examined event.
Traveltimes are computed for an weighted average.       
If radius is set to zero, only the picks of the examied event are used.    

In [ ]:
src['phase_onset'] = {}
src['phase_onset']['src_cluster'] = {'weight':[1000,1], # weights [examined event, event within radius]
                                     'radius':1.0} # radius in km

Define phase windows

In [ ]:
src['phase_onset']['width'] = {'P':[0.5,4.5], # [before,after] P-Onset in sec
                               'S':[0.5,4.5]  # [before,after] S-Onset in sec
                              } 

Load phase onset information

In [ ]:
src['phase_onset']['obs'] =  Load_Picks(Container=cont).get_picks(file_type='cnv',display_tt=True)
#print(src['phase_onset']['obs']['t_travel'].keys())
#print(src['phase_onset']['obs']['t_variance']['9K_AB03'])
print(src['phase_onset']['obs'])

Define phase tabel ID of fomosto synthetics. 

In [ ]:
src['phase_onset']['table_id'] = {'P':'any_P','S':'any_S'} # see config

#### Automated P-phase onset picker    
Pick P-onset time of synthetics.    
This allows you to define a P-phase window when using CAP or 3D synthetics.    
Caution: Use plot tag to check output of STA-LTA Picker!

In [ ]:
pre['synt_onset'] = {}
pre['synt_onset']['perform'] = False
pre['synt_onset']['comp'] = ['Z'] # list of traces which will be used to pick P-onset
pre['synt_onset']['sta_lta'] = [2.0,5.0] # [sta,lta] in seconds
pre['synt_onset']['thresh'] = 0.8 # threshold relative to sta-lta-tseries peak
pre['synt_onset']['plot'] = False#True # plots obspy picker (when running Quick_Loader)


#### Trace time window
Use if phase onset information are not available!    

Define time window relative to reported event time $t_0$ with:                   
t_wind = $[a,b]$               
**a**: time before $t_0$ in s             
**b**: time after $t_0$ in s             
                            
Note: a and b must be within the available duration of the observabels and synthetics
Also, a must be equal or smaller than Pyrocko_t_add or CAP_t_add (lenght of added synthetic zero trace)        
and b equal or smaller than trace_length (pyrocko setting).

In [ ]:
netw['t_wind_synt'] = [0,260]

### Trace selection

Structure of Trace Selection file:
Event_id | obs_trace_set | selection



Example:       

ev123 Set1 XX_DP04-ZRT XX_DP14-ZRT XX_DP18-ZRT ...           
ev123 Set2 XX_DP04-Z XX_DP14-Z ...           
ev143 Set1 XX_DP12-Z XX_DP19-Z XX_DP21-ZR           
...

This routine offers two ways to construct this file:            
**Manual selection**         
**Automatic selection** Old version. Do not use! Needs some bug fixes!    

In [ ]:
# get location of observables
path2data = path['observation_data_path']

# path and name of Trace Selection file 
Trace_selection_file = path2fserver+'/TextFiles/'+region+'/Trace_Selection'
path['Trace_selection_file'] = Trace_selection_file

Signal_window_file = path2fserver+'/TextFiles/'+region+'/Signal_window_Selection'
path['Signal_window_file'] = Signal_window_file

#### Manual selection

In [ ]:
# define intial frequency band
f_init = [0.055,0.075]
# frequency step length (update buttom +-)
df = 0.005


# handeling of information in the Trace Selection file
update_file = 'update' # option: 'update' or 'append' (caution: update overwrites existing selection)
set_tag = 'Set1'#'vertical'#'MarkV'#'fullC'#'vertical'# #  name of selection set

use_tag = set_tag

# define 
print(netw['distance_range'])
distance_range = netw['distance_range'] # stations outside won`t be selected (set to False)
twind = [5,500]#[5,60]#[200,700]#[600,1600]
Pwind = [75,100]
taper = ['hann',0.1]
azimuth_range = [0.,360.]



In [ ]:
#'''
Trace_Selector(path2data=path2data,                   # manditory  
               Station_file=Station_file,             # manditory
               Event_file=Event_file,                 # manditory
               path2file=Trace_selection_file,        # manditory
               path2PWfile=Signal_window_file,        # manditory
               phase_picks=None, # None
               distance_range=distance_range,         # default [0.,1000.]
               azimuth_range=azimuth_range,           # default [0.,360.]
               path2fig='',                           # default pwd
               f_init=f_init,                         # default [0.04,0.06]
               df=df,                                 # default 0.005
               twind=twind,                           # default []
               Pwind=Pwind,                           # default []
               update_file=update_file,               # default a
               set_tag=set_tag,                       # default Set1
               event_id=src['event_id'],              # default evXXX
               taper=taper,                           # default []
               zerophase=False                        # default False
              )      
#'''


In case of forced terimation of Trace_Selector (e.g. X instead of save) run below cell to change back to jupyther notebook backend.

In [ ]:
import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')

#### Automatic selection

Outdated: Do not use or change!

[Obspy Trigger](https://docs.obspy.org/tutorial/code_snippets/trigger_tutorial.html)

In [ ]:

# set list of stations (e.g. [XX_DP01, XX_DP03]) that should be excluded
netw['rm_stations'] = []
#'''
# P-Phase onset picker (used to distinguish between signal and noise)
pre['picker'] = {}
pre['picker']['perform'] = False#True
pre['picker']['filter'] = {'ftype':'bandpass','freqmin':1.0,'freqmax':8.0,'corners':4,'zerophase':True}                      
pre['picker']['STA-LTA'] = [10.0,100.0]
pre['picker']['threshold'] = [4.0,1.5]   
# if perform is false, first x seconds are treated as noise
pre['picker']['t_noise'] = 30 # in s

# signal to noise 
pre['s2n'] = {}
pre['s2n']['window'] = [5,150] # time in sec before and after the onset (picker)
pre['s2n']['s2n_min'] = 3.0 # S2N ratio (in freq band)
pre['H2V'] = [8.0,5.0] # H2V max for [Z,R/T], if measured H2V is below this threshold, the trace is included

# equalizer
pre['equalizer'] = {}
pre['equalizer']['min_coverage'] = 0.5
pre['equalizer']['e_max'] = 0.975
pre['equalizer']['t_uncertainty'] = 0.75

# automated trace selection criteria
pre['selection_criteria'] = {}
pre['selection_criteria']['N_criteria'] = 4 # H2V, S2N, min_coverage, t_uncertainty 
pre['selection_criteria']['stat_min'] = 4
pre['selection_criteria']['trace_min'] = 6
pre['selection_criteria']['perform_envelope_shift'] = False#True#
#'''

### Set trace information

##### Set trace selection set id                           
Options are existing selection ids (e.g. set_tag from manual trace selection) or the automatic modus.           
Set obs_trace_set to 'auto' for automatic selection (only for experienced users)

Get selection dictionary (use print to show content)

In [ ]:
trace_list = selected_traces_from_file(Trace_selection_file,event_list,netw['stat_global'],
                                       distance_range=netw['distance_range'],
                                       rm_stations=netw['rm_stations'],
                                       out=False)

gen['obs_trace_set'] = use_tag # set to auto if selection is automatic
netw['trace_selection'] = trace_list[src['event_id']][gen['obs_trace_set']]

print(netw['trace_selection'])

### Get signal window

Fallback solution if Phase information and not available and full time window contains too much noise.      
Case: Regional network with large variations in source-receiver distance.

In [ ]:
netw['signal_window'] = load_signal_window_file(path['Signal_window_file'])

### Set network information

In [ ]:
STAT = get_Station_information(Container=cont)

#### Source

In [ ]:
# source dictionary (containes source information of major and (optional) minor couple)
src['SRC_dict'] = STAT.src_loc_dict
# source time
src['ref_time'] = STAT.src_time_dict

Magnitude update

In [ ]:
# source magnitude
lit_mag = STAT.src_mag_dict['F1_X0']
print(lit_mag)
perform_mag_update = True#False#
update_Mode = 'P2P' # 'LinReg' # 'AMean'
src['ref_mag'] = [synt_ref_mag,perform_mag_update,lit_mag,update_Mode] 
# perform magnitude update on traces
inv['magnitude_update'] = 'network' #'T'#  vertical, horizontal

#### Network

In [ ]:
# station dictionary (contains station information e.g. location, azimuth, src-distance ...)
netw['STAT_dict'] = STAT.stat_loc_dict
# CAP reference id (distance dependent id, None is Fomosto is used)
netw['CAP_ref_id'] = STAT.CAP_LIST
# CAP file path dictionary 
path['CAP_file_path'] = STAT.CAP_path
# 3D waveform file path dictionary
path['3D_file_path'] = STAT.PATH_3D
#print(path['3D_file_path'])
print(netw['CAP_ref_id'])

## Preprocessing
### Resample

In [ ]:
# resampling (low sampling will increase the computation speed)
pre['resampling'] = {'sampling_rate':1.0, # warning: if set too small issues in tshift may arise
                     'window':'hanning', 
                     'strict_length':False}

### Remove trend

In [ ]:
# detrend
pre['detrend'] = {'type':'demean'}#None}#

### Taper

In [ ]:
# taper
pre['taper'] = {'type':None,#'hann',# set None if no taper should be used
                        'max_percentage':0.1,
                        'max_length':None,
                        'side':'left'}

### Filter

Multifrequency band:                     
define partition type (ufix: upper fixed, lfix: lower fixed or None: use freqmin and freqmax)                 
set fcut list (based on partition)                  

e.g. lfix            
freqmin is fixed at given value             
freqmax is taken from fcut          
lenght of fcut defines number of frequency bands used (Note: for auto selection, selection is also done for the different frequency bands) 

In [ ]:
pre['filter'] = {'ftype':'bandpass',
                'freqmin':0.055,
                'freqmax':0.075,
                'corners':4,
                'zerophase':False,#True,#
                'partition_type':None, # ufix, lfix, None
                'fcut':[0.05,0.06,0.07] 
                        }

#### Doublet filter settings

In [ ]:
pre['doublet_filter'] = {
                        'F1':{
                    'freqmin':0.04,
                    'freqmax':0.07,
                    'corners':4,
                    'zerophase':False#True#
                    },
                    'F2':{
                    'fcut':[0.04],
                    'freqmax':0.07}
                    } 

# uniXtree setting


Parameter vector:           
**ii = 0**: strike of major couple                   
**ii = 1**: dip of major couple                   
**ii = 2**: rake of major couple                   
**ii = 3**: clvd of major couple                   
**ii = 4**: iso of major couple                   
**ii = 5**: strike of minor couple                   
**ii = 6**: dip of minor couple                   
**ii = 7**: rake of minor couple                   
**ii = 8**: clvd of minor couple                   
**ii = 9**: iso of minor couple                   
**ii = 10**: amplitude ration (0.0: 100% major, 1.0: 100% minor)             
**ii = 11**: inter event time in s (>0: tmajor < tminor)                    

Major Couple (F1): ii $\in [0,4]$                 
Minor Couple (F2): ii $\in [5,11]$                 

starting_solution_Fx: initial central location in parameter space                 
Xtree_range_Fx: halfe wide of parameter space               

Caution: set ii=10 and ii=11 to 0.0 for single source simulation   

### Define tree structure

In [ ]:
inv['Xtree'] = {'starting_solution_F1':[180., 45., 0.,0.,0.], 
                 'starting_solution_F2':[180., 45., 0.,0.,0.,0.0,0.0],
                 'Xtree_range_F1':[180.,45.,180.,0.0,0.0],
                 'Xtree_range_F2':[180.,45.,180.,0.0,0.0,0.0,0.0],
                 'full_space_Xlayer': 3, # number of top layer with 100% child volumes
                 'nlay_min':11, #  min number of volume halfing steps
                 'nlay_max':12, # max number of volume halving steps
                 'top_fac':6, # number of top solutions used in next volume halfing
                 'half_min':11, # x = 2**half_min --> 360/x: std octree 
                 'half_max':12,
                 'tcrit':['NL2',10,0.9], # data, n results, threshold --> ai/mean(a(i-1,i-(trci[1]+1)) > trci[2]
                 'multicore':True # use multicore (False is useful for maitanace work in tree structure)
                }

### Cost function
L2: least square             
Bayesian_Cd: data covariance                
Bayesian_CD: data + model covariance (based on solve_for_mis(loc/rot))

In [ ]:
inv['function'] = 'Bayesian_Cd'#'L2' #

#### Calculate cost fonction per 
full_network: classic approach, all traces are add together to a single long time series
per_station: pdf is calculated for each station, use bootstrap to get mean and std

In [ ]:
inv['rec_inv_procedure'] = 'full_network'#'per_station' # 

#### Error normalization

In [ ]:
inv['normalize'] = False # outdated 
inv['L2_normalize'] = True # will be used to compute VR

#### Envelope

In [ ]:
# used in NL2 to include weighted envelope information (0.0: 0% envelope, 1.0: 100% envelope)
inv['envelope_portion'] = 0.0 # float with [0.0,1.0]

#### Bootstrap

In [ ]:
# bootstrap (only availabe for bayesian_inv_procedure = per_station)
inv['bootstrap'] = {}
inv['bootstrap']['perform'] = False#True#
inv['bootstrap']['modus'] =  'trace'#'frequency'#'station'#  TODO: issue with frequency option in station ranking
inv['bootstrap']['factor'] = 0.75
inv['bootstrap']['n'] = 1000

### Time shift

In [ ]:
inv['time_shift'] = {}
inv['time_shift']['perform'] = True # False#
inv['time_shift']['station_Tshift'] = {'cweight':{'Z':1.0,'R':1.0,'T':1.0},
                                      'dt_max_netw':3.0,# network shift --> mean of all traces at all stations
                                      'dt_max_stat':1.0,# station shift --> mean of all traces per station
                                      'dt_max_tr':0.0}  # trace shift   --> each trace is shifted (use with caution)

#### Extract phase window        
Note: Use only with pyrocko and cnv-file

In [ ]:
pre['phase_window'] = {}
pre['phase_window']['perform'] = False#True # 
pre['phase_window']['Taper'] = {'type':None,#'hann',# set None if no taper should be used
                                'max_percentage':0.15,
                                'max_length':None,
                                'side':'both'}

# Postprocessing

### Clustering

In [ ]:
post['mspace_List'] = [5,2.5,5,2.5,2.5,5,2.5,5,2.5,2.5,0.1,2.0] # base variation (std)
post['cmax'] = 50 # max number of cluster
post['rem_fac'] = 0.1 # remaining factor of non clustered solutions

# Output

In [ ]:
plot['waveforms_preproc'] = False#True#
plot['picker'] = False#True#
plot['equalizer']= False#True#

In [ ]:
# specifie path, CAP output will be saved as mseed with global station id 
# at path, six folders for F1 to F6 are produced corresponding to the fundamental source mechanism
# for no output set path to None
write['CAP_output'] = None# '/home/mlind/Python_Scripts/Final_Code/Data/Karibik/Synthetics_6Fund/'#
write['3D_output'] = None

## Create optional result dictionary

In [ ]:
res['LinInv'] = {}
res['GridInv'] = {}
res['TreeInv'] = {}

## Create json

In [ ]:
with open(pwd+'/input_'+src['event_id']+'_'+region+'.json', 'w', encoding='utf-8') as f:
    json.dump(cont, f, ensure_ascii=False, indent=4)
print('input_'+src['event_id']+'_'+region+'.json')